In [1]:
from IPython.core.interactiveshell import InteractiveShell  #执行该代码可以使得当前nb支持多输出
InteractiveShell.ast_node_interactivity = "all" 
import numpy as np
from datetime import datetime
import pandas as pd 
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False 
from mpl_toolkits.mplot3d import Axes3D

import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, LSTM
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math


pd.options.display.max_rows = 8  
%matplotlib inline

In [5]:
import tushare as ts
#df = ts.shibor_data() #取当前年份的数据
df = ts.shibor_data(2014) #取2014年的数据
# df.sort('date', ascending=False).head(10)
print(df)
# for i in range(2016,2023):
#     df = ts.shibor_data(i) #取2014年的数据
#     df.sort('date', ascending=False)



None


In [11]:
import tushare as ts
ts.set_token('a7b16f2ba6fc460fbb8126d87948c66627c7550aead54c8fa6eb1ba4')
pro = ts.pro_api()
df = pro.shibor(start_date='20160121', end_date='20220221')
df = df.sort_values('date', ascending=True)
df.to_csv("data/shibor.csv",encoding='gbk',index = True)

In [24]:
national_debt = pd.read_csv('data/national_debt_5y.csv',encoding = 'gb2312')  # 读取股票文件
national_debt['date']= pd.to_datetime(national_debt['日期'])
national_debt.head()

df = df.reset_index(drop = True)
df['date'] = pd.to_datetime(df['date'])
df.head()

len(df)
len(national_debt)

,日期,开盘,收盘,高,低,涨跌幅,date
0,2016/1/21,2.683,2.700,2.703,2.680,0.78%,2016-01-21
1,2016/1/22,2.705,2.689,2.705,2.682,-0.41%,2016-01-22
2,2016/1/25,2.750,2.694,2.750,2.680,0.19%,2016-01-25
3,2016/1/26,2.703,2.738,2.740,2.690,1.63%,2016-01-26
4,2016/1/27,2.763,2.774,2.783,2.763,1.31%,2016-01-27


,date,on,1w,2w,1m,3m,6m,9m,1y
0,2016-01-21,2.014,2.405,2.812,2.918,2.9860,3.1030,3.1740,3.2530
1,2016-01-22,2.028,2.400,2.836,2.990,3.0310,3.1130,3.1780,3.2560
2,2016-01-25,2.009,2.376,2.843,3.019,3.0415,3.1180,3.1800,3.2590
3,2016-01-26,1.997,2.363,2.900,3.057,3.0730,3.1235,3.1842,3.2682
4,2016-01-27,1.995,2.359,2.931,3.092,3.1040,3.1210,3.1810,3.2650


1497

1563

In [25]:
data = pd.merge(national_debt,df,left_on = 'date',right_on = 'date',how = 'left')
data
data = data.fillna(method = 'ffill')

data = data[['日期','开盘','收盘','高','低','涨跌幅','on']]
data.index= data['日期']
data.to_csv("data/new.csv",encoding='gbk',index = False)

,日期,开盘,收盘,高,低,涨跌幅,date,on,1w,2w,1m,3m,6m,9m,1y
0,2016/1/21,2.683,2.700,2.703,2.680,0.78%,2016-01-21,2.014,2.405,2.812,2.918,2.9860,3.1030,3.1740,3.2530
1,2016/1/22,2.705,2.689,2.705,2.682,-0.41%,2016-01-22,2.028,2.400,2.836,2.990,3.0310,3.1130,3.1780,3.2560
2,2016/1/25,2.750,2.694,2.750,2.680,0.19%,2016-01-25,2.009,2.376,2.843,3.019,3.0415,3.1180,3.1800,3.2590
3,2016/1/26,2.703,2.738,2.740,2.690,1.63%,2016-01-26,1.997,2.363,2.900,3.057,3.0730,3.1235,3.1842,3.2682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1559,2022/2/16,2.559,2.500,2.559,2.498,-0.48%,2022-02-16,1.942,2.057,2.028,2.340,2.4100,2.4660,2.5400,2.6040
1560,2022/2/17,2.526,2.482,2.526,2.478,-0.72%,2022-02-17,1.855,2.002,2.047,2.334,2.4060,2.4630,2.5340,2.5960
1561,2022/2/18,2.511,2.505,2.511,2.479,0.93%,2022-02-18,2.110,2.098,2.227,2.323,2.4000,2.4570,2.5240,2.5860
1562,2022/2/21,2.500,2.506,2.520,2.497,0.00%,2022-02-21,2.195,2.140,2.302,2.315,2.3920,2.4530,2.5150,2.5790
